### Problem Statement:
School Scheduling is a **MILP** problem.
Right after the Covid lockdown lift, schools were allowed to operate at only 50% room capacity. The problem is to help schools figure out how to schedule with a lot of constraints stated below:

Cary Elementary has 18 classrooms and 12 grades and populations and capacity is provided in SRO_input_XX.csv files. There are 8 one-hour blocks of time where students can have face-to-face instruction. Decide the optimum schedule for each student grade to maximize total aggregated student face-to-face time while not allowing any grade have more face-to-face instruction time than other grades. Also, no consecutive time slots are allowed.

In [1]:
import pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import math

In [2]:
# Read the input files
input_grades_df = pd.read_csv("data/SRO_input_grades.csv")
input_rooms_df = pd.read_csv("data/SRO_input_rooms.csv")

In [3]:
# Understanding the data
print(input_rooms_df)
print(input_grades_df)

   Room_ID  Capacity
0     A121        15
1     A120        13
2     A123        15
3     A137        12
4     A138        15
5     B142        12
6     B144        12
7     B146        12
8     B148        12
9     C153        12
10    C155        19
11    C156        12
12    C157        13
13    D162        13
14    D163        13
15      T1        40
16      T2        40
17      T3        40
         Grade_ID  Population
0           PreKA          12
1           PreKB          14
2           PreKC          10
3   KindergartenA          16
4   KindergartenB          18
5          FirstA          15
6          FirstB          16
7         SecondA          32
8          ThirdA          16
9          ThirdB          17
10        FourthA          35
11         FifthA          37


### Layman Formulation:

##### DECISION VARIABLE:

For each grade, each hour, and each classroom, we have a decision variable (Asgn) that represents whether students of that grade will have face-to-face instruction in that hour and classroom (1) or not (0).

##### CONSTRAINTS:

##### Fair Assignment Rule:
The total face-to-face instruction time for each grade should be balanced. No grade should have significantly more instruction time than others.

##### One Grade Per Class Slot Rule:
In each hour and classroom, only one grade can have face-to-face instruction. This ensures that classrooms are not double-booked.

##### One Room Per Grade Slot Rule:
In each hour, a grade can have face-to-face instruction in only one classroom. This prevents a grade from occupying multiple classrooms simultaneously.

##### Grade Fits Room Rule:
The number of students in a grade attending face-to-face instruction in a particular hour and classroom should not exceed the room's capacity.

##### No Consecutive Classes Rule
Any grade should not have consecutive classes.

##### OBJECTIVE FUNCTION:

The objective is to maximize the total face-to-face instruction time for all students. In simple terms, we want to make the best use of available resources and time to ensure students get the most in-person learning experience.

In [9]:
m = AbstractModel()

# Define sets
m.GRADES = Set()
m.HOURS = Set()
m.ROOMS = Set()

# Define Parameters
m.pop = Param(m.GRADES)
m.cap = Param(m.ROOMS)

# Define var
m.Asgn = Var(m.GRADES, m.HOURS, m.ROOMS, domain = Binary)

# Define constraints
def Fair_Assignment_Rule(m, g1, g2):
    if g1>g2:
        return sum(m.Asgn[g1, h, r] for h in m.HOURS for r in m.ROOMS) == sum(m.Asgn[g2, h, r] for h in m.HOURS for r in m.ROOMS)
    else:
        return Constraint.Skip
m.Fair_Assignment = Constraint(m.GRADES, m.GRADES, rule=Fair_Assignment_Rule)

def One_Grade_PerClassSlot_Rule(m, h, r):
    return sum(m.Asgn[g, h, r] for g in m.GRADES) <= 1
m.One_Grade_PerClassSlot = Constraint(m.HOURS, m.ROOMS, rule = One_Grade_PerClassSlot_Rule)

def One_Room_Per_GradeSlot_Rule(m, g, h):
    return sum(m.Asgn[g, h, r] for r in m.ROOMS) <= 1
m.One_Room_Per_GradeSlot = Constraint(m.GRADES, m.HOURS, rule = One_Room_Per_GradeSlot_Rule)

def Grade_Fits_Room_Rule(m, g, h, r):
    return m.pop[g] * m.Asgn[g, h, r] <= m.cap[r]
m.Grade_Fits_Room = Constraint(m.GRADES, m.HOURS, m.ROOMS, rule = Grade_Fits_Room_Rule)

def No_Consecutive_Classes_Rule(m, g, h):
    # Check if the current hour is not the last hour in the set
    if h < max(m.HOURS):
        # Ensure that the current hour and the next hour do not both have assignments
        return sum(m.Asgn[g, h, r] + m.Asgn[g, h + 1, r] for r in m.ROOMS) <= 1
    else:
        # If the current hour is the last hour, no need to check for consecutive classes
        return Constraint.Skip
m.No_Consecutive_Classes = Constraint(m.GRADES, m.HOURS, rule=No_Consecutive_Classes_Rule)

def Total_Time_Rule(m):
    return sum(m.Asgn[g, h, r] for g in m.GRADES for h in m.HOURS for r in m.ROOMS)
m.Total_Time = Objective(rule = Total_Time_Rule, sense = maximize)

In [10]:
instanceData = {None:{
    'GRADES': {None: list(input_grades_df.index)},
    'HOURS': {None: list(range(1, 9))},
    'ROOMS': {None: input_rooms_df['Room_ID']},
    'pop': input_grades_df['Population'].to_dict(),
    'cap': input_rooms_df.set_index(['Room_ID']).to_dict()['Capacity']
}}

# Build instance
instance = m.create_instance(instanceData)

In [11]:
solver = SolverFactory('glpk')
#Solve
sol = solver.solve(instance)
print(sol['Solver'])


- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.3258650302886963



In [12]:
instance.Total_Time.expr()

48.0

In [13]:
# Extract schedule information
schedule_data = {'Time Slot': [], 'Room': [], 'Grade': []}

for g in instance.GRADES:
    for h in instance.HOURS:
        for r in instance.ROOMS:
            if value(instance.Asgn[g, h, r]) > 0:  # Check if the assignment is made
                schedule_data['Time Slot'].append(h)
                schedule_data['Room'].append(r)
                schedule_data['Grade'].append(input_grades_df.loc[g, 'Grade_ID'])

# Create a DataFrame
schedule_df = pd.DataFrame(schedule_data)

# Pivot the DataFrame for better visibility
schedule_pivot = schedule_df.pivot_table(index='Grade', columns='Time Slot', values='Room', aggfunc='first')

# Display the schedule DataFrame
schedule_pivot

Time Slot,1,2,3,4,5,6,7,8
Grade,,,,,,,,
FifthA,NaN,T3,NaN,T3,NaN,T3,NaN,T3
FirstA,A138,NaN,A121,NaN,A123,NaN,A123,NaN
FirstB,NaN,T2,NaN,C155,NaN,C155,NaN,C155
FourthA,NaN,T1,NaN,T1,NaN,T1,NaN,T1
KindergartenA,C155,NaN,T2,NaN,C155,NaN,T3,NaN
KindergartenB,T2,NaN,C155,NaN,T2,NaN,T2,NaN
PreKA,A121,NaN,A120,NaN,A137,NaN,A121,NaN
PreKB,A123,NaN,A123,NaN,A121,NaN,A138,NaN
PreKC,A120,NaN,A137,NaN,A120,NaN,A120,NaN
